# Лабораторная работа 6

   по теме
                 
   **Прогноз успеха фильмов по обзорам**
   
   ****

   Выполнил студент

   Группы БСТ1801

   Харатишвили Заза

In [151]:
%matplotlib inline
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.utils import to_categorical
from keras import models
from keras import layers
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.models import Sequential

In [152]:
# Загружаем датасет
from keras.datasets import imdb
(training_data, training_targets), (testing_data,testing_targets) = imdb.load_data(num_words=10000)
# Обьединяем данные в доль одной оси
data = np.concatenate((training_data, testing_data), axis=0)
targets = np.concatenate((training_targets, testing_targets),axis=0)

In [153]:
print("Категории:", np.unique(targets)) # Положительный или отрицательный обзоры
print("Количество уникальных слов:", len(np.unique(np.hstack(data))))
length = [len(i) for i in data]
print("Средний размер обзора:", np.mean(length))
print("Стандартное отклонение:", round(np.std(length)))

Категории: [0 1]
Количество уникальных слов: 9998
Средний размер обзора: 234.75892
Стандартное отклонение: 173


In [155]:
# Посмотрим на первый обзор из датасета
print("Номер:", targets[0])
print(data[0])

Номер: 1
[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 2, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 2, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 2, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [121]:
# Преобразуем индексы в слова
index = imdb.get_word_index()
reverse_index = dict([(value, key) for (key, value) in index.items()])
decoded = " ".join( [reverse_index.get(i - 3, "#") for i in data[0]] ) # Заменим неизвестные слова на #
print(decoded)

# this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert # is an amazing actor and now the same being director # father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for # and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also # to the two little boy's that played the # of norman and paul they were just brilliant children are often left out of the # list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you thi

In [156]:
# Подготовим данные
# Заполним нулями каждый обзор, чтобы размер был 10000 чисел
# Это необходимо, потому что каждый элемент входных данных должен иметь одинакоый размер

def vectorize(sequences, dimension = 10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1
    return results

data = vectorize(data)
print(tf.shape(data))

targets = np.array(targets).astype("float32") # Приведем к типу float32


tf.Tensor([50000 10000], shape=(2,), dtype=int32)


In [157]:
# Разделим датасет на обучающий и тестировочный наборы. Обучающий 40 000 обзоров, тестовый — 10 000.
test_x = data[:10000]
test_y = targets[:10000]
train_x = data[10000:]
train_y = targets[10000:]

print(tf.shape(test_x))
print(tf.shape(test_y))

tf.Tensor([10000 10000], shape=(2,), dtype=int32)
tf.Tensor([10000], shape=(1,), dtype=int32)


In [200]:
# Создадим и обучим модель
model = Sequential()
model.add(layers.Dense(50, activation = "relu", input_shape=(10000, )))
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))
model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))
model.add(layers.Dense(1, activation = "sigmoid"))
model.summary()

# Компилируем модель
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["acc"])

results = model.fit(train_x, train_y, epochs= 2, batch_size = 32, validation_data = (test_x, test_y))

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_44 (Dense)            (None, 50)                500050    
                                                                 
 dropout_22 (Dropout)        (None, 50)                0         
                                                                 
 dense_45 (Dense)            (None, 50)                2550      
                                                                 
 dropout_23 (Dropout)        (None, 50)                0         
                                                                 
 dense_46 (Dense)            (None, 50)                2550      
                                                                 
 dense_47 (Dense)            (None, 1)                 51        
                                                                 
Total params: 505,201
Trainable params: 505,201
Non-t

In [201]:
# Проведем оценку работы модели
print("Точность обучения:", np.mean(results.history["val_acc"]))

Точность обучения: 0.8911499977111816


Напишем функцию иморта пользовательского текстового файла

In [185]:
from nltk.tokenize import RegexpTokenizer

In [186]:
def tokenize_words(input):
    # lowercase everything to standardize it
    input = input.lower()

    # instantiate the tokenizer
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(input)

    # if the created token isn't in the stop words, make it part of "filtered"
    # filtered = filter(lambda token: token not in stopwords.words('english'), tokens)
    return " ".join(tokens)

In [187]:
file = open("text.txt",'r').read()

processed_inputs = tokenize_words(file)
processed_inputs = processed_inputs.replace("ђ", "")
processed_inputs = processed_inputs.replace("њ", "")
processed_inputs = processed_inputs.replace("ќ", "")
processed_inputs = processed_inputs.replace("в", "")

chars = sorted(list(set(processed_inputs)))
char_to_num = dict((c, i) for i, c in enumerate(chars))

print(processed_inputs)

input_len = len(processed_inputs)
vocab_len = len(chars)
print ("Total number of characters:", input_len)
print ("Total vocab:", vocab_len)

video games as perhaps the most popular form of entertainment nowadays is currently suffering from the same problem surprisingly in the times when you needed to insert a magnetic tape cassette into your zx spectrum to play anything games could offer more thrill and fun than today not because games were better but mostly because the ideas they implemented were still new
Total number of characters: 371
Total vocab: 24


In [192]:
seq_length = 1
x_data = []
y_data = []

# loop through inputs, start at the beginning and go until we hit
# the final character we can create a sequence out of
for i in range(0, input_len - seq_length, 1):
    # Define input and output sequences
    # Input is the current character plus desired sequence length
    in_seq = processed_inputs[i:i + seq_length]

    # Out sequence is the initial character plus total sequence length
    out_seq = processed_inputs[i + seq_length]

    # We now convert list of characters to integers based on
    # previously and add the values to our lists
    x_data.append([char_to_num[char] for char in in_seq])
    y_data.append(char_to_num[out_seq])

n_patterns = len(x_data)
print ("Total Patterns:", n_patterns)

X = np.reshape(x_data, (n_patterns, seq_length))
X = X/float(vocab_len)

def vectorize(sequences, dimension = 10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1
    return results
y_data = vectorize(y_data)

print(tf.shape(y_data))
print(tf.shape(X))

y_data = np.array(y_data).astype("float32") # Приведем к типу float32

Total Patterns: 370
tf.Tensor([  370 10000], shape=(2,), dtype=int32)
tf.Tensor([370   1], shape=(2,), dtype=int32)


In [199]:
data_loss, data_accuracy = model.evaluate(y_data, X, verbose=0)
print('test acc:', data_loss)

test acc: 0.685989499092102


## Вывод

В результате выполнения лабораторной работы можно сделать вывод о том, что 